In [ ]:
pip install python-telegram-bot==20.0.0
pip install nest_asyncio
pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install -qqq bitsandbytes==0.43.1 trl peft==0.10.0 accelerate==0.28.0
!pip install -qqq transformers==4.39.3
!pip install -qqq flash-attn --no-build-isolation
!pip install -qqq accelerate bitsandbytes -i https://pypi.org/simple/

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig

<h3>PreTrained

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

<h3>Llm

In [2]:
MODEL_NAME = "medicalai/ClinicalGPT-base-zh"

def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model_llm = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model_llm, tokenizer

model_llm, tokenizer = create_model_and_tokenizer()
model_llm.config.use_cache = False

# LoRA configuration
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "k_proj"],
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00002-of-00016.safetensors:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

model-00003-of-00016.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

model-00004-of-00016.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

...

model-00016-of-00016.safetensors:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
import nest_asyncio
import asyncio
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters
import os
from PIL import Image
from transformers import AutoImageProcessor, DetrForObjectDetection
import torch
import logging
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, CallbackQueryHandler, ContextTypes
import re

In [ ]:
TOKEN = 'The_bot_token'
deases_type = ""

# Apply nest_asyncio to avoid event loop issues
nest_asyncio.apply()

# Function to handle the /start command
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = [
        [InlineKeyboardButton("اعاني من السكر", callback_data='diabetes')],
        [InlineKeyboardButton("اعاني من مرض في القلب", callback_data='heart disease')],
        [InlineKeyboardButton("اعاني من الضغط", callback_data='Decompression sickness')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text('Please choose an option:', reply_markup=reply_markup)

# Function to handle button clicks
async def button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()
    choice = query.data # catch the choose
    await query.edit_message_text(text=f"thank you for clarifications\nPlease send me a food image :)")

    # save the name of deases in the variable => deases_type
    global deases_type
    deases_type = choice


# Pretrained model and Llm functions
def pretrained_model(image_url):
    logging.getLogger("transformers").setLevel(logging.ERROR)

    image = Image.open(image_url)

    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

    objects = set()

    for label in results["labels"]:
        objects.add(model.config.id2label[label.item()])

    # Start Llm work
    food_type = "and ".join(list(objects)) if len(objects) > 1 else str(list(objects)[0])
    global deases_type

    # Prompt
    input_text = f"""
    You a doctor recieved a name of some foods from one of your patient.
    You advice him to eat or not based on the names of items \
    speak as a doctor.

    ### deases: ```{deases_type}```

    ### food: ```{food_type}```

    ### question: Is the items good for people who suffer from a this deases?

    ### Answer:
    """

    inputs = tokenizer(input_text, return_tensors="pt")
    output = model_llm.generate(**inputs, max_length=200+(len(food_type)+len(deases_type))*50) # size of output based on the count of deases and food
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # filter the output
    pattern = r"### Answer:\s*(.*)"

    match = re.search(pattern, decoded_output, re.DOTALL)
    if match:
        decoded_output = match.group(1).strip()
    else:
        decoded_output = ""

    return(decoded_output)


# Welcome Message
async def welcome_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(
        chat_id=update.effective_chat.id,
        text='Hi, I am a bot to help you to check if you can eat this food or not.\nTo start, click on /start.'
    )


# Read image then call models
async def process_image(photo_name: str):
    name, f_ext = os.path.splitext(photo_name)
    data_image = pretrained_model(f'./temp/{photo_name}')
    return data_image


# Take the image then call process_image function to work on the image that taken form the user
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if filters.PHOTO.check_update(update):
        file_id = update.message.photo[-1].file_id
        unique_file_id = update.message.photo[-1].file_unique_id
        photo_name = f'{unique_file_id}.jpg'
    elif filters.Document.IMAGE.check_update(update):
        file_id = update.message.document.file_id
        _, f_ext = os.path.splitext(update.message.document.file_name)
        unique_file_id = update.message.document.file_unique_id
        photo_name = f'{unique_file_id}.{f_ext}'

    photo_file = await context.bot.get_file(file_id)
    await photo_file.download_to_drive(custom_path=f'./temp/{photo_name}')
    await context.bot.send_message(chat_id=update.effective_chat.id, text='We are processing your photo. Please wait!')
    final_text = await process_image(photo_name)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=final_text)
    os.remove(f'./temp/{photo_name}')


# Main build for the bot
async def main():
    application = ApplicationBuilder().token(TOKEN).build()

    welcome_handler = MessageHandler(filters.StatusUpdate.NEW_CHAT_MEMBERS | filters.ChatType.PRIVATE, welcome_message)
    button_handler = CallbackQueryHandler(button)
    start_handler = CommandHandler('start', start)
    message_handler = MessageHandler(filters.PHOTO | filters.Document.IMAGE, handle_message)

    application.add_handler(message_handler)
    application.add_handler(button_handler)
    application.add_handler(start_handler)
    application.add_handler(welcome_handler)

    await application.run_polling()

# Run the bot
await main()  # Use this line if you're in an async environment